# Results Analysis

1. Best combinaison






In [32]:
import pandas as pd
import pickle as pk

prediction = pd.read_csv(f"outputs/cities_prediction.csv")
common_columns = ["Geoname ID", "Name", "ASCII Name", "Alternate Names", "Feature Class", "Feature Code", "Country Code", "Country name EN", "Country Code 2", "Admin1 Code", "Admin2 Code", "Admin3 Code", "Admin4 Code", "Population", "Elevation", "DIgital Elevation Model", "Timezone", "Modification date", "LABEL EN", "Coordinates", "city"]


## 1. Best / Worst combinaisons

Distance min/max/average by combinaison

In [34]:
import numpy as np

# Compute mean, standard deviation, and count of non-NaN values for columns containing "distance"
distance_stats = {
    k: (prediction[k].mean(), prediction[k].std(), prediction[k].count()) 
    for k in prediction.keys() if "distance" in k
}

# Sort columns by their mean value
sorted_columns = sorted(distance_stats.items(), key=lambda item: item[1][0])

# Select the 5 best (lowest mean) and 5 worst (highest mean) columns
best_5 = sorted_columns[:5]   # Top 5 best (lowest mean)
worst_5 = sorted_columns[-5:] # Top 5 worst (highest mean)

# Display results
print("🏆 **Top 5 Best Columns** (Lowest Mean):")
for col, (mean, std, count) in best_5:
    print(f"{col} | Mean: {mean:.4f} | STD: {std:.4f} | Non-NaN values: {count}")

print("\n💀 **Top 5 Worst Columns** (Highest Mean):")
for col, (mean, std, count) in reversed(worst_5):  # Reverse to show the worst first
    print(f"{col} | Mean: {mean:.4f} | STD: {std:.4f} | Non-NaN values: {count}")


🏆 **Top 5 Best Columns** (Lowest Mean):
mistralai/Mistral-Small-24B-Base-2501_float16_gps_fr_distance | Mean: 87.0070 | STD: 672.1141 | Non-NaN values: 904
meta-llama/Llama-3.2-3B_int4_gps_fr_distance | Mean: 88.6713 | STD: 187.2747 | Non-NaN values: 38
meta-llama/Llama-2-13b-hf_int8_gps_fr_distance | Mean: 108.8562 | STD: 168.5298 | Non-NaN values: 35
mistralai/Mistral-Small-24B-Base-2501_int8_gps_fr_distance | Mean: 113.4336 | STD: 830.8944 | Non-NaN values: 792
Qwen/Qwen2.5-14B_float16_gps_fr_distance | Mean: 114.9181 | STD: 653.1495 | Non-NaN values: 984

💀 **Top 5 Worst Columns** (Highest Mean):
meta-llama/Llama-3.1-8B-Instruct_float16_gps_fr_distance | Mean: 6290.6920 | STD: 5321.7150 | Non-NaN values: 607
meta-llama/Llama-2-13b-chat-hf_float16_gps_fr_distance | Mean: 6127.1839 | STD: 4790.6469 | Non-NaN values: 971
meta-llama/Llama-3.1-8B-Instruct_int8_gps_fr_distance | Mean: 6050.3589 | STD: 5063.5476 | Non-NaN values: 757
meta-llama/Llama-2-13b-chat-hf_int8_gps_fr_distance | M

**Remove columns when there is not enough distance computed (~750)**

In [28]:
df = prediction.copy()
df = df.dropna(axis=1, thresh=len(df) - 250)

distance_stats = {
    k: (df[k].mean(), df[k].std(), df[k].count()) 
    for k in df.keys() if "distance" in k
}

# Sort columns by their mean value
sorted_columns = sorted(distance_stats.items(), key=lambda item: item[1][0])

# Select the 5 best (lowest mean) and 5 worst (highest mean) columns
best_5 = sorted_columns[:5]   # Top 5 best (lowest mean)
worst_5 = sorted_columns[-5:] # Top 5 worst (highest mean)

# Display results
print("🏆 **Top 5 Best Columns** (Lowest Mean):")
for col, (mean, std, count) in best_5:
    print(f"{col} | Mean: {mean:.4f} | STD: {std:.4f} | Non-NaN values: {count}")

print("\n💀 **Top 5 Worst Columns** (Highest Mean):")
for col, (mean, std, count) in reversed(worst_5):  # Reverse to show the worst first
    print(f"{col} | Mean: {mean:.4f} | STD: {std:.4f} | Non-NaN values: {count}")

🏆 **Top 5 Best Columns** (Lowest Mean):

💀 **Top 5 Worst Columns** (Highest Mean):


In [30]:
df

,Geoname ID,Name,ASCII Name,Alternate Names,Feature Class,Feature Code,Country Code,Country name EN,Admin1 Code,Admin2 Code,...,Admin4 Code,Population,DIgital Elevation Model,Timezone,Modification date,LABEL EN,Coordinates,city,Latitude,Longitude
0,2988507,Paris,Paris,"Baariis,Bahliz,Baris,Lungsod ng Paris,Lutece,L...",P,PPLC,FR,France,11,75,...,75056,2138551,42,Europe/Paris,2023-09-22,France,"48.85341, 2.3488",Paris,48.85341,2.34880
1,2995469,Marseille,Marseille,"MRS,Marseggia,Marsegia,Marseglia,Marseilla,Mar...",P,PPLA,FR,France,93,13,...,13055,870731,28,Europe/Paris,2022-08-14,France,"43.29695, 5.38107",Marseille,43.29695,5.38107
2,2996944,Lyon,Lyon,"LYS,Leon do Roine,León do Roine,Lijon,Lio,Lion...",P,PPLA,FR,France,84,69,...,69123,522969,173,Europe/Paris,2022-12-27,France,"45.74846, 4.84671",Lyon,45.74846,4.84671
3,2972315,Toulouse,Toulouse,"Lapangan Terbang Blagnac,TLS,Tolosa,Tolosa Okz...",P,PPLA,FR,France,76,31,...,31555,493465,150,Europe/Paris,2022-08-14,France,"43.60426, 1.44367",Toulouse,43.60426,1.44367
4,2990440,Nice,Nice,"NCE,Nica,Nicaea,Nicc,Nicca,Niccae,Nice,Nicea,N...",P,PPLA2,FR,France,93,06,...,06088,342669,18,Europe/Paris,2023-08-11,France,"43.70313, 7.26608",Nice,43.70313,7.26608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2987003,Pithiviers,Pithiviers,"Pithiviers,Pitiv'e,Pitive,pi di wei ye,pitivui...",P,PPLA3,FR,France,24,45,...,45252,9659,121,Europe/Paris,2019-04-10,France,"48.17185, 2.25185",Pithiviers,48.17185,2.25185
1996,3022826,Coutances,Coutances,"Constantia,Couotaunches,Coutances,Koutans,Kuta...",P,PPLA3,FR,France,28,50,...,50147,9657,69,Europe/Paris,2019-04-10,France,"49.04541, -1.44518",Coutances,49.04541,-1.44518
1997,3002172,Les Andelys,Les Andelys,"Andeliacum,Les Andelys,Lez Andeli,Lez-Andeli,l...",P,PPLA3,FR,France,28,27,...,27016,9655,18,Europe/Paris,2019-04-10,France,"49.24557, 1.41168",Les Andelys,49.24557,1.41168
1998,3019865,Erstein,Erstein,"Eerstain,Erstein,Ersten,ai er si tan,arstyn,er...",P,PPL,FR,France,44,67,...,67130,9647,156,Europe/Paris,2016-02-18,France,"48.42373, 7.66262",Erstein,48.42373,7.66262


In [36]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Sélectionner les colonnes contenant "distance"
distance_columns = [col for col in prediction.columns if "distance" in col]

# Calculer la moyenne des distances pour chaque colonne
distance_means = [prediction[col].mean() for col in distance_columns]

# Créer un DataFrame avec ces moyennes et les critères
df = pd.DataFrame({
    "distance_mean": distance_means,
    "model_family": [col.split("_")[0] for col in distance_columns],
    "model_size": [col.split("_")[1] for col in distance_columns],
    "quantization": [col.split("_")[2] for col in distance_columns],
    "lang_prompt": [col.split("_")[3] for col in distance_columns],
    "prompt_type": [col.split("_")[4] for col in distance_columns],
    # "fine_tuning": [col.split("_")[5] for col in distance_columns]
})

# ANOVA à plusieurs facteurs avec un modèle linéaire
# formula = "distance_mean ~ C(model_family) + C(model_size) + C(quantization) + C(lang_prompt) + C(prompt_type) + C(fine_tuning)"
formula = "distance_mean ~ C(model_family) + C(model_size) + C(quantization) + C(lang_prompt) + C(prompt_type)"
model = smf.ols(formula, data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)  # ANOVA Type II

# Afficher les résultats
print("📊 **ANOVA Results on Means of Distances:**")
print(anova_table)

# Identifier le critère le plus significatif (plus petite p-value)
most_significant = anova_table["PR(>F)"].idxmin()
print(f"\n✅ The most significant factor is: **{most_significant}** (p-value = {anova_table.loc[most_significant, 'PR(>F)']:.4e})")


ValueError: must have at least one row in constraint matrix